# TAT-C: Collect Observations Example

This example demonstrates how to use direct function calls of the low-level TAT-C library to collect observations of a surface point.

First we define the satellites for a mission. In this example we define one satellite (NOAA-20) with a single instrument (Visual/Infrared Imager Radiometer Suite, VIIRS) and an orbit derived from a known two-line element set (TLE). The TLE defines the orbital state at an instant in time which can be used to determine future positions of the satellite. The example TLE below for NOAA-20 was collected from [CelesTrak](https://celestrak.org/) around July 2022.

TAT-C allows several types of orbit specifications, one of which being the `TwoLineElements` specification which requires the TLE as an argument.

In [1]:
noaa20_tle = [
    "1 43013U 17073A   22195.78278435  .00000038  00000+0  38919-4 0  9996",
    "2 43013  98.7169 133.9110 0001202  63.8768 296.2532 14.19561306241107",
]

from tatc.schemas import TwoLineElements

noaa20_orbit = TwoLineElements(tle=noaa20_tle)

Next, we define an instrument to perform observations. The most important parameter is the _field of regard_ which sets the view angle from the satellite to the Earth's surface. The TAT-C `utils` package contains a function to calculate the field of regard (111.6 degrees) from an instrument's stated swath width (3000 km) and altitude (834 km).

In [2]:
from tatc import utils

viirs_for = utils.swath_width_to_field_of_regard(834000, 3000000)
print(f"computed field of regard: {viirs_for:.1f}")

from tatc.schemas import Instrument

viirs = Instrument(name="VIIRS", field_of_regard=viirs_for)

computed field of regard: 111.6


Finally, we can combine the orbit and instrument to define a new satellite.

In [3]:
from tatc.schemas import Satellite

noaa20 = Satellite(name="NOAA 20", orbit=noaa20_orbit, instruments=[viirs])

Next, we define the point of interest to observe using geodetic coordinates.

In [4]:
from tatc.schemas import Point

hoboken = Point(id=0, latitude=40.74259, longitude=-74.02686)

Next, we can identify the starting and ending time of a sample mission period. The starting time is noon UTC on July 14, 2022 and the ending time is 30 days later (noon UTC on August 13, 2022).

In [5]:
from datetime import datetime, timedelta, timezone

start = datetime(year=2022, month=7, day=14, hour=12, tzinfo=timezone.utc)
end = start + timedelta(days=30)

The `collect_observations` analysis function identifies all of the observation opportunities of a point by a satellite instrument between the starting and ending time. Results are formatted as a flat GeoDataFrame which is similar to a regular pandas DataFrame with a geospatial column labeled `geometry`. Other columns:
 * `start`: observation rise time (or mission start for first observation)
 * `end`: observation set time (or mission end for last observation)
 * `epoch`: observation midpoint
 * `sat_alt`: satellite altitude angle (degrees) at epoch
 * `sat_az`: satellite azimuth angle (degrees) at epoch

In [6]:
from tatc.analysis import collect_observations

results = collect_observations(hoboken, noaa20, viirs, start, end)
display(results)

,point_id,geometry,satellite,instrument,start,end,epoch,sat_alt,sat_az
0,0,POINT (-74.02686 40.74259),NOAA 20,VIIRS,2022-07-14 17:14:31.249456+00:00,2022-07-14 17:21:09.247941+00:00,2022-07-14 17:17:50.248698500+00:00,45.831953,69.758229
1,0,POINT (-74.02686 40.74259),NOAA 20,VIIRS,2022-07-14 18:56:58.592616+00:00,2022-07-14 18:59:29.582181+00:00,2022-07-14 18:58:14.087398500+00:00,22.556942,266.371762
2,0,POINT (-74.02686 40.74259),NOAA 20,VIIRS,2022-07-15 07:12:52.816811+00:00,2022-07-15 07:19:53.419438+00:00,2022-07-15 07:16:23.118124500+00:00,54.469723,288.858738
3,0,POINT (-74.02686 40.74259),NOAA 20,VIIRS,2022-07-15 16:56:31.348293+00:00,2022-07-15 17:01:48.037784+00:00,2022-07-15 16:59:09.693038500+00:00,31.730816,66.710747
4,0,POINT (-74.02686 40.74259),NOAA 20,VIIRS,2022-07-15 18:36:30.316720+00:00,2022-07-15 18:42:04.528667+00:00,2022-07-15 18:39:17.422693500+00:00,33.134265,263.480643
...,...,...,...,...,...,...,...,...,...
82,0,POINT (-74.02686 40.74259),NOAA 20,VIIRS,2022-08-11 16:50:16.726627+00:00,2022-08-11 16:54:50.675332+00:00,2022-08-11 16:52:33.700979500+00:00,28.100283,65.603930
83,0,POINT (-74.02686 40.74259),NOAA 20,VIIRS,2022-08-11 18:29:31.987104+00:00,2022-08-11 18:35:39.567890+00:00,2022-08-11 18:32:35.777497+00:00,37.905952,262.511909
84,0,POINT (-74.02686 40.74259),NOAA 20,VIIRS,2022-08-12 06:47:14.649777+00:00,2022-08-12 06:54:43.896264+00:00,2022-08-12 06:50:59.273020500+00:00,89.432083,76.599537
85,0,POINT (-74.02686 40.74259),NOAA 20,VIIRS,2022-08-12 18:10:09.428091+00:00,2022-08-12 18:17:16.455789+00:00,2022-08-12 18:13:42.941940+00:00,56.555333,259.878779


The `aggregate_observations` function groups observations of points to compute metrics like access and revisit.
 * `access`: duration of an observation
 * `revisit`: duration since the prior observation (note: the first observation has `NaT` for missing data

In [7]:
from tatc.analysis import aggregate_observations

aggregated_results = aggregate_observations(results)
display(aggregated_results)

,point_id,geometry,satellite,instrument,start,epoch,end,access,revisit
0,0,POINT (-74.02686 40.74259),NOAA 20,VIIRS,2022-07-14 17:14:31.249456+00:00,2022-07-14 17:17:50.248698624+00:00,2022-07-14 17:21:09.247941+00:00,0 days 00:06:37.998485,NaT
1,0,POINT (-74.02686 40.74259),NOAA 20,VIIRS,2022-07-14 18:56:58.592616+00:00,2022-07-14 18:58:14.087398400+00:00,2022-07-14 18:59:29.582181+00:00,0 days 00:02:30.989565,0 days 01:35:49.344675
2,0,POINT (-74.02686 40.74259),NOAA 20,VIIRS,2022-07-15 07:12:52.816811+00:00,2022-07-15 07:16:23.118124544+00:00,2022-07-15 07:19:53.419438+00:00,0 days 00:07:00.602627,0 days 12:13:23.234630
3,0,POINT (-74.02686 40.74259),NOAA 20,VIIRS,2022-07-15 16:56:31.348293+00:00,2022-07-15 16:59:09.693038592+00:00,2022-07-15 17:01:48.037784+00:00,0 days 00:05:16.689491,0 days 09:36:37.928855
4,0,POINT (-74.02686 40.74259),NOAA 20,VIIRS,2022-07-15 18:36:30.316720+00:00,2022-07-15 18:39:17.422693376+00:00,2022-07-15 18:42:04.528667+00:00,0 days 00:05:34.211947,0 days 01:34:42.278936
...,...,...,...,...,...,...,...,...,...
82,0,POINT (-74.02686 40.74259),NOAA 20,VIIRS,2022-08-11 16:50:16.726627+00:00,2022-08-11 16:52:33.700979456+00:00,2022-08-11 16:54:50.675332+00:00,0 days 00:04:33.948705,0 days 09:36:54.295453
83,0,POINT (-74.02686 40.74259),NOAA 20,VIIRS,2022-08-11 18:29:31.987104+00:00,2022-08-11 18:32:35.777497088+00:00,2022-08-11 18:35:39.567890+00:00,0 days 00:06:07.580786,0 days 01:34:41.311772
84,0,POINT (-74.02686 40.74259),NOAA 20,VIIRS,2022-08-12 06:47:14.649777+00:00,2022-08-12 06:50:59.273020416+00:00,2022-08-12 06:54:43.896264+00:00,0 days 00:07:29.246487,0 days 12:11:35.081887
85,0,POINT (-74.02686 40.74259),NOAA 20,VIIRS,2022-08-12 18:10:09.428091+00:00,2022-08-12 18:13:42.941939968+00:00,2022-08-12 18:17:16.455789+00:00,0 days 00:07:07.027698,0 days 11:15:25.531827


Finally, TAT-C includes a supplemental method `reduce_observations` to reduce observations to descriptive statistics.

In [8]:
from tatc.analysis import reduce_observations

reduced_results = reduce_observations(aggregated_results)
display(reduced_results)

,point_id,geometry,access,revisit,samples
0,0,POINT (-74.02686 40.74259),0 days 00:05:52.132532,0 days 08:08:57.677311,87


The same sequence of operations can be repeated for satellite constellations using the `collect_multi_observations` function call.

In [ ]:
from tatc.schemas import WalkerConstellation

const = WalkerConstellation(
    name="NOAA 20",
    orbit=noaa20_orbit,
    instruments=[viirs],
    number_satellites=3,
    number_planes=3,
    configuration="star",
)

from tatc.analysis import collect_multi_observations

results = collect_multi_observations(hoboken, const, start, end)
display(results)

In [ ]:
aggregated_results = aggregate_observations(results)
display(aggregated_results)

In [ ]:
reduced_results = reduce_observations(aggregated_results)
display(reduced_results)

Enabling quick sensitivity analyses.

In [ ]:
import numpy as np

n = np.arange(1, 7)
revisit = [
    reduce_observations(
        aggregate_observations(
            collect_multi_observations(
                hoboken,
                WalkerConstellation(
                    name="NOAA 20",
                    orbit=noaa20_orbit,
                    instruments=[viirs],
                    number_satellites=i,
                    number_planes=i,
                    configuration="star",
                ),
                start,
                end,
            )
        )
    )
    .iloc[0]
    .revisit
    / timedelta(hours=1)
    for i in n
]

import matplotlib.pyplot as plt

plt.figure()
plt.plot(n, revisit, ".-")
plt.xlabel("Number Satellites/Planes (Walker Star)")
plt.ylabel("Mean Revisit (hr)")
plt.show()